# Proyecto Bimestral: Sistema de Recuperaci´on de Información basado en Reuters-21578
## 1. Introducción
El objetivo de este proyecto es diseñar, construir, programar y desplegar un Sistema de Recuperación de Información (SRI) utilizando el corpus Reuters-21578.
## 2. Fases del Proyecto
###  2.1. Adquisición de datos

*   Descargar el Corpus Reuters-21578
*   Descomprimir y organizar los archivos
*   Documentar el proceso de adquisición de datos

![Descripción de la imagen](images/corpus_reuters.jpg)

#### Descarga y análisis inicial del corpus

1. El archivo comprimido fue descargado y descomprimido en una carpeta local vinculada a un repositorio de GitHub.
2. Posteriormente, se analizó el contenido del corpus, obteniendo los siguientes resultados:
   - **Carpeta `test`**: Contiene 3019 archivos.
   - **Carpeta `training`**: Contiene 7769 archivos.
   - **Archivo `cats`**: Incluye las categorías.
   - **Archivo `readme`**: Proporciona información general sobre el corpus.
   - **Archivo `stopwords`**: Contiene una lista de palabras vacías.

#### Librerias y Dependencias necesarias

#### Notas

Al no encontrarnos en un entorno de Google Colab, sino en VS, nos basta con ejecutar una sola vez el comando `!pip install rarfile` para tener la biblioteca `rarfile` en nuestro entorno.

In [27]:
#!pip install rarfile

Al no encontrarnos en un entorno de Google Colab, sino en VS, nos basta con ejecutar una sola vez el comando `!pip install nltk` para tener la biblioteca `nltk` en nuestro entorno.

In [29]:
#!pip install nltk

El comando `pip install scikit-learn` instala la librería `scikit-learn`, que es una herramienta para realizar tareas de aprendizaje automático (machine learning) como clasificación, regresión y clustering.

In [17]:
#pip install scikit-learn

El comando `pip install gensim` instala la librería `gensim`, que se utiliza para el procesamiento de texto y la creación de modelos de aprendizaje automático, como Word2Vec, para generar representaciones vectoriales de palabras.

In [19]:
#pip install gensim

In [1]:
# Librerias necesarias
import os
import re
import nltk
import rarfile
import time # Evaluar tecnicas de vectorización
import psutil # Evaluar tecnicas de vectorización
import gc # Evaluar tecnicas de vectorización
import numpy as np
from collections import defaultdict # Para el indice invertido
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

### 2.2. Preprocesamiento

#### 2.2.1. Extraer el contenido relevante de los documentos

In [19]:
rarfile.UNRAR_TOOL = r"D:\UnRAR\UnRAR.exe" # Cambiar por ubicación local de tu herramienta UNRAR

# Ruta del archivo .rar
rar_path = 'material/reuters.rar'
output_dir = 'material/content'

# Crear el directorio de salida si no existe
os.makedirs(output_dir, exist_ok=True)

# Descomprimir el archivo .rar
with rarfile.RarFile(rar_path) as rf:
    rf.extractall(output_dir)

# Verificar que se haya descomprimido correctamente
print("Archivos extraídos:")
print(os.listdir(output_dir))

Archivos extraídos:
['reuters']


In [2]:
# Directorios de documentos
train_dir = 'material/content/reuters/training'
test_dir = 'material/content/reuters/test'
cats_file = 'material/content/reuters/cats.txt'

# Diccionario para almacenar documentos
documentos = {}

# Función para extraer contenido de un archivo de noticias
def extraer_texto(filepath):
    try:
        with open(filepath, 'r', encoding='latin-1') as file:
            contenido = file.read()
            texto_limpio = contenido.strip()  # Elimina espacios en blanco iniciales y finales
            return texto_limpio
    except Exception as e:
        print(f"Error al leer el archivo {filepath}: {e}")
        return ""

# Función para cargar las categorías de los documentos
def cargar_categorias(filepath):
    categorias = {}
    try:
        with open(filepath, 'r', encoding='latin-1') as file:
            for linea in file:
                partes = linea.strip().split()  # Divide la línea en partes separadas por espacios
                if len(partes) >= 2:
                    doc_id = partes[0]  # Primer elemento es el id del documento (test/14826, training/1)
                    etiquetas = partes[1:]  # Resto de elementos son categorías
                    categorias[doc_id] = etiquetas
    except Exception as e:
        print(f"Error al leer el archivo de categorías {filepath}: {e}")
    return categorias

# Función para cargar los documentos y asociar categorías
def cargar_documentos(directorio, tipo, categorias_dict):
    archivos = os.listdir(directorio)
    if not archivos:
        print(f"No se encontraron archivos en {directorio}")
    for archivo in archivos:
        filepath = os.path.join(directorio, archivo)
        doc_id = f"{tipo}/{archivo}"
        texto = extraer_texto(filepath)
        categorias = categorias_dict.get(doc_id, [])  # Obtener categorías, si no hay, devuelve lista vacía

        # Almacenar en el diccionario
        documentos[doc_id] = {
            "texto": texto,
            "categorias": categorias
        }

In [5]:
# Cargar categorías
categorias_dict = cargar_categorias(cats_file)

# Cargar documentos de entrenamiento y prueba
cargar_documentos(train_dir, 'training', categorias_dict)
cargar_documentos(test_dir, 'test', categorias_dict)

# Verificar el tamaño y algunas muestras
print(f"Total de documentos cargados: {len(documentos)}")
if documentos:
    ejemplo_doc = list(documentos.items())[0]
    print(f"ID: {ejemplo_doc[0]}")
    print(f"Texto: {ejemplo_doc[1]['texto'][:500]}...")  # Mostrar los primeros 500 caracteres
    print(f"Categorías: {ejemplo_doc[1]['categorias']}")
else:
    print("No se cargaron documentos.")

Total de documentos cargados: 10788
ID: training/1
Texto: BAHIA COCOA REVIEW
  Showers continued throughout the week in
  the Bahia cocoa zone, alleviating the drought since early
  January and improving prospects for the coming temporao,
  although normal humidity levels have not been restored,
  Comissaria Smith said in its weekly review.
      The dry period means the temporao will be late this year.
      Arrivals for the week ended February 22 were 155,221 bags
  of 60 kilos making a cumulative total for the season of 5.93
  mln against 5.81 at th...
Categorías: ['cocoa']


#### 2.2.2. Realizar limpieza de datos: eliminación de caracteres no deseados, normalización de texto, etc.

In [3]:
# Función para limpiar texto
def limpiar_texto(texto):
    # 1. Conversión a minúsculas
    texto = texto.lower()

    # 2. Eliminación de caracteres especiales y números
    texto = re.sub(r'[^a-z\s]', '', texto)

    # 3. Eliminación de espacios extra
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto

In [6]:
# Aplicar limpieza de texto a todos los documentos
for doc_id in documentos:
    texto_original = documentos[doc_id]['texto']
    texto_limpio = limpiar_texto(texto_original)

    # Actualizar el texto limpio en el diccionario
    documentos[doc_id]['texto'] = texto_limpio

# Verificar el resultado de la limpieza en un documento de ejemplo
ejemplo_doc = list(documentos.items())[0]
print(f"ID: {ejemplo_doc[0]}")
print(f"Texto limpio: {ejemplo_doc[1]['texto'][:500]}...")

ID: training/1
Texto limpio: bahia cocoa review showers continued throughout the week in the bahia cocoa zone alleviating the drought since early january and improving prospects for the coming temporao although normal humidity levels have not been restored comissaria smith said in its weekly review the dry period means the temporao will be late this year arrivals for the week ended february were bags of kilos making a cumulative total for the season of mln against at the same stage last year again it seems that cocoa delive...


#### 2.2.3. Tokenización: dividir el texto en palabras o tokens

#### Nota
Al no encontrarnos en un entorno de Google Colab, sino en VS, nos basta con ejecutar una sola vez el comando `nltk.download('punkt')` o `nltk.download('punkt_tab')`, por lo cual no es necesario volver a ejcutarlo.

In [46]:
# Descargar los recursos necesarios de NLTK para tokenizar texto en oraciones y palabras
#nltk.download('punkt')
#nltk.download('punkt_tab')

In [7]:
# Función para tokenizar texto
def tokenizar_texto(texto):
    # Utilizamos word_tokenize de NLTK para dividir en tokens
    tokens = word_tokenize(texto)
    return tokens

In [8]:
# Aplicar tokenización a todos los documentos
for doc_id in documentos:
    texto_limpio = documentos[doc_id]['texto']
    tokens = tokenizar_texto(texto_limpio)

    # Guardamos los tokens en el diccionario
    documentos[doc_id]['tokens'] = tokens

# Verificar el resultado de la tokenización en un documento de ejemplo
ejemplo_doc = list(documentos.items())[0]
print(f"ID: {ejemplo_doc[0]}")
print(f"Tokens: {ejemplo_doc[1]['tokens'][:20]}...")

ID: training/1
Tokens: ['bahia', 'cocoa', 'review', 'showers', 'continued', 'throughout', 'the', 'week', 'in', 'the', 'bahia', 'cocoa', 'zone', 'alleviating', 'the', 'drought', 'since', 'early', 'january', 'and']...


#### 2.2.4. Eliminar stop words y  aplicar stemming o lematización.

In [9]:
# Cargar el archivo de stopwords proporcionado
ruta_stopwords = 'material/content/reuters/stopwords'
stopwords_personalizadas = set()

with open(ruta_stopwords, 'r') as archivo:
    for linea in archivo:
        palabra = linea.strip()  # Removemos espacios y saltos de línea
        if palabra:  # Evitamos añadir líneas vacías
            stopwords_personalizadas.add(palabra.lower())

# Verificamos algunas stop words cargadas
print("Ejemplo de Stop Words cargadas:", list(stopwords_personalizadas)[:10])

Ejemplo de Stop Words cargadas: ['just', 'goes', 'overall', 'why', 'went', 'somewhat', 'plus', "what's", 'are', 'everywhere']


In [10]:
# Función para eliminar stop words de los tokens
def eliminar_stopwords(tokens, stopwords):
    # Filtramos los tokens que no están en la lista de stopwords
    tokens_filtrados = [token for token in tokens if token.lower() not in stopwords]
    return tokens_filtrados

# Aplicamos la eliminación de stop words a cada documento
for doc_id in documentos:
    tokens = documentos[doc_id]['tokens']
    tokens_sin_stopwords = eliminar_stopwords(tokens, stopwords_personalizadas)

    # Guardamos los tokens filtrados
    documentos[doc_id]['tokens'] = tokens_sin_stopwords

# Verificar el resultado después de eliminar stop words en un documento de ejemplo
ejemplo_doc = list(documentos.items())[0]
print(f"ID: {ejemplo_doc[0]}")
print(f"Tokens sin Stop Words: {ejemplo_doc[1]['tokens'][:20]}...")

ID: training/1
Tokens sin Stop Words: ['bahia', 'cocoa', 'review', 'showers', 'continued', 'week', 'bahia', 'cocoa', 'zone', 'alleviating', 'drought', 'early', 'january', 'improving', 'prospects', 'coming', 'temporao', 'normal', 'humidity', 'levels']...


#### Nota
Para nuestro caso de estudio y aplicación de este proyecto, aplicaremos **Steming**. 

Como equipo, consideramos más eficiente en términos de recursos porque el stemming utiliza reglas más simples y rápidas para reducir las palabras a su raíz, mientras que la lemmatización requiere un procesamiento más complejo y el uso de diccionarios para obtener la forma base correcta de las palabras.

In [11]:
# Inicializar el stemmer
stemmer = PorterStemmer()

# Función para aplicar stemming a los tokens de un documento
def aplicar_stemming(tokens):
    return [stemmer.stem(token) for token in tokens]

In [12]:
# Aplicamos el stemming a los tokens de cada documento
for doc_id in documentos:
    tokens = documentos[doc_id]['tokens']  # Obtiene los tokens del documento
    tokens_stemmed = aplicar_stemming(tokens)  # Aplica el stemming
    documentos[doc_id]['tokens'] = tokens_stemmed  # Guarda los tokens procesados

# Verificamos un documento de ejemplo después del stemming
ejemplo_doc = list(documentos.items())[0]
print(f"ID: {ejemplo_doc[0]}")
print(f"Tokens con Stemming: {ejemplo_doc[1]['tokens'][:20]}...")

ID: training/1
Tokens con Stemming: ['bahia', 'cocoa', 'review', 'shower', 'continu', 'week', 'bahia', 'cocoa', 'zone', 'allevi', 'drought', 'earli', 'januari', 'improv', 'prospect', 'come', 'temporao', 'normal', 'humid', 'level']...


### 2.3.  Representación de Datos en Espacio Vectorial

#### 2.3.1. Utilizar técnicas como Bag of Words (BoW), TF-IDF, y Word2Vec.

In [13]:
# Crear un objeto de CountVectorizer
vectorizer = CountVectorizer()

# Obtener las palabras de los documentos (usando los textos limpios después de eliminar stopwords y stemming)
documentos_texto = [" ".join(doc["tokens"]) for doc in documentos.values()]

# Ajustar el modelo y transformar los documentos
X_bow = vectorizer.fit_transform(documentos_texto)

# Mostrar las características (palabras)
print(f"Características (Palabras) BoW: {vectorizer.get_feature_names_out()[:10]}...")  # Mostrar las primeras 10 palabras

# Mostrar la matriz de términos de frecuencia
print(f"Forma de la matriz BoW: {X_bow.shape}")

Características (Palabras) BoW: ['aa' 'aaa' 'aabex' 'aachen' 'aaminu' 'aancor' 'aap' 'aaplu' 'aar'
 'aarnoud']...
Forma de la matriz BoW: (10788, 26004)


In [14]:
# Crear el objeto TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Ajustar y transformar los documentos
X_tfidf = tfidf_vectorizer.fit_transform(documentos_texto)

# Mostrar las características (palabras)
print(f"Características (Palabras) TF-IDF: {tfidf_vectorizer.get_feature_names_out()[:10]}...")  # Primeras 10 palabras

# Mostrar la matriz de TF-IDF
print(f"Forma de la matriz TF-IDF: {X_tfidf.shape}")

Características (Palabras) TF-IDF: ['aa' 'aaa' 'aabex' 'aachen' 'aaminu' 'aancor' 'aap' 'aaplu' 'aar'
 'aarnoud']...
Forma de la matriz TF-IDF: (10788, 26004)


In [15]:
# Entrenamiento del modelo Word2Vec
model_w2v = Word2Vec(sentences=[doc["tokens"] for doc in documentos.values()], vector_size=100, window=5, min_count=1, workers=4)

# Función para obtener el vector medio de un documento
def obtener_vector_promedio(tokens, model):
    # Extraer los vectores de las palabras, promediarlos y retornar el vector
    vectores = [model.wv[token] for token in tokens if token in model.wv]
    if vectores:
        return np.mean(vectores, axis=0)
    else:
        return np.zeros(model.vector_size)  # Devuelve un vector cero si no hay palabras del vocabulario en el modelo

# Obtener los vectores para cada documento
documentos_w2v = {}
for doc_id, doc in documentos.items():
    vector_promedio = obtener_vector_promedio(doc['tokens'], model_w2v)
    documentos_w2v[doc_id] = vector_promedio

# Verificar el tamaño de los vectores
print(f"Dimensiones del vector Word2Vec: {documentos_w2v[next(iter(documentos_w2v))].shape}")

Dimensiones del vector Word2Vec: (100,)


#### 2.3.2. Evaluar las diferentes técnicas de vectorización.

In [16]:
# Función para medir el tiempo y la memoria
def medir_tiempo_memoria(func):
    # Medir el tiempo de ejecución
    start_time = time.time()
    
    # Medir la memoria antes de ejecutar
    proceso = psutil.Process()
    memoria_inicial = proceso.memory_info().rss  # Memoria en bytes
    
    # Ejecutar la función
    result = func()
    
    # Medir el tiempo y la memoria después de ejecutar
    end_time = time.time()
    memoria_final = proceso.memory_info().rss
    memoria_utilizada = memoria_final - memoria_inicial
    
    # Calcular tiempo de ejecución
    tiempo_ejecucion = end_time - start_time
    
    return tiempo_ejecucion, memoria_utilizada, result

# Evaluar BoW (Bag of Words)
def evaluar_bow():
    vectorizer = CountVectorizer()
    X_bow = vectorizer.fit_transform(documentos_texto)
    return X_bow

# Evaluar TF-IDF
def evaluar_tfidf():
    tfidf_vectorizer = TfidfVectorizer()
    X_tfidf = tfidf_vectorizer.fit_transform(documentos_texto)
    return X_tfidf

# Evaluar Word2Vec
def evaluar_word2vec():
    model_w2v = Word2Vec(sentences=[doc["tokens"] for doc in documentos.values()], vector_size=100, window=5, min_count=1, workers=4)
    def obtener_vector_promedio(tokens, model):
        vectores = [model.wv[token] for token in tokens if token in model.wv]
        if vectores:
            return np.mean(vectores, axis=0)
        else:
            return np.zeros(model.vector_size)
    documentos_w2v = {doc_id: obtener_vector_promedio(doc['tokens'], model_w2v) for doc_id, doc in documentos.items()}
    return documentos_w2v

# Evaluar cada técnica
tiempo_bow, memoria_bow, _ = medir_tiempo_memoria(evaluar_bow)
tiempo_tfidf, memoria_tfidf, _ = medir_tiempo_memoria(evaluar_tfidf)
tiempo_w2v, memoria_w2v, _ = medir_tiempo_memoria(evaluar_word2vec)

# Mostrar los resultados
print(f"Evaluación BoW: Tiempo = {tiempo_bow:.4f} segundos, Memoria = {memoria_bow / (1024 * 1024):.2f} MB")
print(f"Evaluación TF-IDF: Tiempo = {tiempo_tfidf:.4f} segundos, Memoria = {memoria_tfidf / (1024 * 1024):.2f} MB")
print(f"Evaluación Word2Vec: Tiempo = {tiempo_w2v:.4f} segundos, Memoria = {memoria_w2v / (1024 * 1024):.2f} MB")

# Limpiar memoria
gc.collect()

Evaluación BoW: Tiempo = 0.9270 segundos, Memoria = 7.77 MB
Evaluación TF-IDF: Tiempo = 0.9977 segundos, Memoria = 3.73 MB
Evaluación Word2Vec: Tiempo = 6.8307 segundos, Memoria = 4.65 MB


0

#### Conclusión de la Evaluación

- **BoW** es la técnica más rápida en términos de tiempo de ejecución y requiere una cantidad moderada de memoria para procesar los documentos. Por lo tanto, es la opción más eficiente en tiempo y memoria, ideal para aplicaciones que necesitan rapidez.
- **TF-IDF**, aunque es un poco más lento que BoW y consume algo más de memoria, ofrece un buen equilibrio entre rendimiento y precisión. Esto se debe a que calcula pesos adicionales para las palabras, lo que incrementa el tiempo de procesamiento y la memoria.
- **Word2Vec**, a pesar de usar menos memoria, tiene un tiempo de ejecución significativamente mayor. Esto se debe a que entrena un modelo de vectores de palabras, lo que es más costoso computacionalmente. Es más lento, pero resulta más robusto y preciso, especialmente cuando se necesita capturar relaciones semánticas entre las palabras.


### 2.4.  Indexación 

#### 2.4.1.  Construir un Índice invertido que mapee términos a documentos.

In [17]:
# Crear el índice invertido
def construir_indice_invertido(documentos):
    indice_invertido = defaultdict(set)
    
    # Recorremos cada documento
    for doc_id, doc in documentos.items():
        # Recorrer cada token del documento
        for token in doc["tokens"]:
            indice_invertido[token].add(doc_id)
    
    return indice_invertido

In [18]:
# Construir el índice invertido
indice_invertido = construir_indice_invertido(documentos)

# Mostrar el índice invertido para las primeras palabras
for i, (term, docs) in enumerate(indice_invertido.items()):
    if i >= 1:  # Limitar a las primeras 1 entradas para no saturar la salida
        break
    print(f"Término: '{term}' -> Documentos: {docs}")

Término: 'bahia' -> Documentos: {'training/11459', 'test/17568', 'training/1', 'test/15580', 'test/16071', 'training/11911', 'training/13462'}


### Interpretación

Con el **Índice Invertido** cada línea de la salida muestra un término (como `'bahia'` o `'cocoa'`) y los documentos (con identificadores como `'training/11459'`) en los que dicho término aparece.

#### 2.4.2.  Implementar y optimizar estructuras de datos para el Índice.

In [19]:
# Función para medir el tiempo y la memoria de la construcción del índice
def medir_tiempo_memoria_indice(func, documentos):
    start_time = time.time()
    
    # Medir la memoria antes de ejecutar
    proceso = psutil.Process()
    memoria_inicial = proceso.memory_info().rss
    
    # Ejecutar la función para construir el índice
    indice = func(documentos)
    
    # Medir el tiempo y la memoria después de ejecutar
    end_time = time.time()
    memoria_final = proceso.memory_info().rss
    memoria_utilizada = memoria_final - memoria_inicial
    
    tiempo_ejecucion = end_time - start_time
    
    return tiempo_ejecucion, memoria_utilizada, indice

# Evaluar el índice invertido
tiempo_indice, memoria_indice, indice = medir_tiempo_memoria_indice(construir_indice_invertido, documentos)

# Mostrar resultados
print(f"Tiempo para construir el índice invertido: {tiempo_indice:.4f} segundos")
print(f"Memoria utilizada: {memoria_indice / (1024 * 1024):.2f} MB")

# Mostrar algunos términos en el índice invertido para verificar
for termino, docs in list(indice.items())[:1]:  # Muestra los primeros 5 términos
    print(f"Término: '{termino}' -> Documentos: {docs}")

Tiempo para construir el índice invertido: 0.3150 segundos
Memoria utilizada: 27.08 MB
Término: 'bahia' -> Documentos: {'training/11459', 'test/17568', 'training/1', 'test/15580', 'test/16071', 'training/11911', 'training/13462'}


### 2.5. Diseño del Motor de Búsqueda

#### 2.5.1.  Desarrollar la lógica para procesar consultas de usuarios.

In [20]:
# Procesar una consulta utilizando las funciones de limpieza y tokenización
def procesar_consulta(consulta, indice_invertido):
    # Limpieza de la consulta
    consulta_limpia = limpiar_texto(consulta)
    
    # Tokenización de la consulta
    consulta_tokens = tokenizar_texto(consulta_limpia)
    
    # Inicializar un conjunto para los documentos relevantes
    documentos_relevantes = set()
    
    # Buscar cada término en el índice invertido
    for termino in consulta_tokens:
        if termino in indice_invertido:
            documentos_relevantes.update(indice_invertido[termino])
    
    return documentos_relevantes

In [32]:
# Ejemplo de uso
consulta_usuario = "bahia"  # Ejemplo de consulta del usuario
documentos_encontrados = procesar_consulta(consulta_usuario, indice_invertido)

print(f"Documentos que contienen los términos de la consulta: {documentos_encontrados}")

Documentos que contienen los términos de la consulta: {'training/11459', 'training/1', 'test/15580', 'test/17568', 'test/16071', 'training/11911', 'training/13462'}


#### 2.5.2. Utilizar algoritmos de similitud como similitud coseno o Jaccard.

In [33]:
# Función para calcular similitud de coseno
def similitud_coseno(consulta, documentos):
    vectorizador = TfidfVectorizer()
    
    # Extraer textos de los documentos
    textos_documentos = [documentos[doc_id]['texto'] for doc_id in documentos]
    
    # Crear el corpus con la consulta y los textos relevantes
    corpus = [consulta] + textos_documentos
    
    # Generar la matriz TF-IDF
    matriz_tfidf = vectorizador.fit_transform(corpus)
    
    # Calcular la similitud de coseno entre la consulta y cada documento
    similitudes = cosine_similarity(matriz_tfidf[0:1], matriz_tfidf[1:]).flatten()
    
    # Retornar resultados como un diccionario
    return dict(zip(documentos.keys(), similitudes))

# Función para calcular similitud de Jaccard
def similitud_jaccard(consulta, documentos):
    tokens_consulta = set(tokenizar_texto(limpiar_texto(consulta)))
    similitudes = {}
    
    for doc_id, doc in documentos.items():
        tokens_doc = set(doc['tokens'])
        interseccion = len(tokens_consulta & tokens_doc)
        union = len(tokens_consulta | tokens_doc)
        similitudes[doc_id] = interseccion / union if union > 0 else 0
    
    return similitudes

In [40]:
# Proceso principal
consulta_usuario = "bahia"  # Consulta de ejemplo
documentos_encontrados_ids = procesar_consulta(consulta_usuario, indice_invertido)

# Filtrar los documentos encontrados
documentos_encontrados = {doc_id: documentos[doc_id] for doc_id in documentos_encontrados_ids}

if documentos_encontrados:
    print("Similitud de Coseno:")
    resultados_coseno = similitud_coseno(consulta_usuario, documentos_encontrados)
    for doc, sim in resultados_coseno.items():
        print(f"{doc}: {sim:.4f}")
    
    print("\nSimilitud de Jaccard:")
    resultados_jaccard = similitud_jaccard(consulta_usuario, documentos_encontrados)
    for doc, sim in resultados_jaccard.items():
        print(f"{doc}: {sim:.4f}")
else:
    print("No se encontraron documentos relevantes para la consulta.")

Similitud de Coseno:
training/11459: 0.0368
training/1: 0.0552
test/15580: 0.0687
test/17568: 0.0346
test/16071: 0.0706
training/11911: 0.1087
training/13462: 0.0067

Similitud de Jaccard:
training/11459: 0.0100
training/1: 0.0079
test/15580: 0.0370
test/17568: 0.0204
test/16071: 0.0233
training/11911: 0.0345
training/13462: 0.0058


#### 2.5.2. Desarrollar un algoritmo de ranking para ordenar los resultados.

In [ ]:
def rankear_documentos(resultados_coseno, resultados_jaccard):
    # Rankear documentos por similitud de coseno
    documentos_rankeados_coseno = sorted(resultados_coseno.items(), key=lambda x: x[1], reverse=True)

    # Rankear documentos por similitud de Jaccard
    documentos_rankeados_jaccard = sorted(resultados_jaccard.items(), key=lambda x: x[1], reverse=True)

    return documentos_rankeados_coseno, documentos_rankeados_jaccard

In [43]:
# Aplicar el algoritmo de ranking
documentos_rankeados_coseno, documentos_rankeados_jaccard = rankear_documentos(resultados_coseno, resultados_jaccard)

# Mostrar documentos ordenados por similitud de Coseno
print("Documentos ordenados por similitud de Coseno:")
for doc, puntaje in documentos_rankeados_coseno:
    print(f"{doc}: {puntaje:.4f}")

# Mostrar documentos ordenados por similitud de Jaccard
print("\nDocumentos ordenados por similitud de Jaccard:")
for doc, puntaje in documentos_rankeados_jaccard:
    print(f"{doc}: {puntaje:.4f}")

Documentos ordenados por similitud de Coseno:
training/11911: 0.1087
test/16071: 0.0706
test/15580: 0.0687
training/1: 0.0552
training/11459: 0.0368
test/17568: 0.0346
training/13462: 0.0067

Documentos ordenados por similitud de Jaccard:
test/15580: 0.0370
training/11911: 0.0345
test/16071: 0.0233
test/17568: 0.0204
training/11459: 0.0100
training/1: 0.0079
training/13462: 0.0058


#### Razonamienta pesos de similitud coseno y jaccard

- **Coseno (0.7):** Se le da más peso porque mide relaciones contextuales y distribucionales, especialmente útil para documentos largos.
- **Jaccard (0.3):** Es útil para medir la presencia exacta de términos, pero en general es más limitado en su aplicabilidad comparado con la similitud de Coseno.

### 2.6. Diseño del Motor de Búsqueda

#### 2.6.1.  Definir un conjunto de métricas de evaluación (precisión, recall, F1-score).

#### 2.6.2.  Realizar pruebas utilizando el conjunto de prueba del corpus.

#### 2.6.3.   Comparar el rendimiento de diferentes configuraciones del sistema.